# Spam Classifier
## Assignment Preamble
Please ensure you carefully read all of the details and instructions on the assignment page, this section, and the rest of the notebook. If anything is unclear at any time please post on the forum or ask a tutor well in advance of the assignment deadline.

In addition to all of the instructions in the body of the assignment below, you must also follow the following technical instructions for all assignments in this unit. *Failure to do so may result in a grade of zero.*
* [At the bottom of the page](#Submission-Test) is some code which checks you meet the submission requirements. You **must** ensure that this runs correctly before submission.
* Do not modify or delete any of the cells that are marked as test cells, even if they appear to be empty.
* Do not duplicate any cells in the notebook – this can break the marking script. Instead, insert a new cell (e.g. from the menu) and copy across any contents as necessary.
* Do not use global variables or rely on global state at all. It may lose you marks in the marking script.

Remember to save and backup your work regularly, and double-check you are submitting the correct version.

This notebook is the primary reference for your submission. You may write code in separate `.py` files but it must be clearly imported into the notebook so that it runs without needing to reference those files, and you must explain clearly what functionality is contained in those files (through comments, markdown cells, etc).

As always, **the work you submit for this assignment must be entirely your own.** Do not copy or work with other students. Do not copy answers that you find online. These assignments are designed to help improve your understanding first and foremost – the process of doing the assignment is part of *learning*. They are also used to assess your ability, and so you must uphold academic integrity. Submitting plagiarised work risks your entire place on your degree.

**The pass mark for this assignment is 40%.** We expect that students, on average, will be able to produce a submission which gets a mark between 50-70% within the normal workload allocation for the unit, but this will vary depending on individual backgrounds. Please ask for help if you are struggling.

## Getting Started
Spam refers to unwanted email, often in the form of advertisements. In the literature, an email that is **not** spam is called *ham*. Most email providers offer automatic spam filtering, where spam emails will be moved to a separate inbox based on their contents. Of course this requires being able to scan an email and determine whether it is spam or ham, a classification problem. This is the subject of this assignment.

This assignment has one part, worth 100% of the grade for this coursework.

You will write a supervised learning based classifier to determine whether a given email is spam or ham. You must write and submit the code in this notebook. The training data is provided for you. You may use any classification method. Marks will be awarded primarily based on the accuracy of your classifier on unseen test data, but there are also marks for estimating how accurate you think your classifier will be.

### Choice of Algorithm
While the classification method is a completely free choice, the assignment folder includes [a separate notebook file](data/naivebayes.ipynb) which can help you implement a Naïve Bayes solution. If you do use this notebook, you are still responsible for porting your code into *this* notebook for submission. A good implementation should give a high  enough accuracy to get a good grade on this section (50-70%).

You could also consider a k-nearest neighbour algorithm, but this may be less accurate. Logistic regression is another option that you may wish to consider.

If you are looking to go beyond the scope of the unit, you might be interested in building something more advanced, like an artificial neural network. This is possible just using `numpy`, but will require significant self-directed learning. *Extensions like this are left unguided and are not factored into the unit workload estimates.*

**Note:** you may use helper functions in libraries like `numpy` or `scipy`, but you **must not** import code which builds entire models for you. This includes but is not limited to use of libraries like `scikit-learn`, `tensorflow`, or `pytorch` – there will be plenty of opportunities for these libraries in later units. The point of this assignment is to understand code the actual algorithm yourself. ***If you are in any doubt about any particular library or function please ask a tutor.*** Submissions which ignore this will receive penalties or even zero marks.

## Training Data
The training data is described below and has 1000 rows. There is also a 500 row set of test data. These are functionally identical to the training data, they are just in a separate csv file to encourage you to split out your training and test data. You should consider how to best make use of all available data without overfitting, and to help produce an unbiased estimate for your classifier's accuracy.

The cell below loads the training data into a variable called `training_spam`.

In [1]:
import numpy as np

training_spam = np.loadtxt(open("data/training_spam.csv"), delimiter=",").astype(np.int)
print("Shape of the spam training data set:", training_spam.shape)
print(training_spam)

Shape of the spam training data set: (1000, 55)
[[1 0 0 ... 0 0 0]
 [0 0 1 ... 1 0 0]
 [0 0 0 ... 1 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [1 1 1 ... 1 1 0]
 [1 0 0 ... 1 1 1]]


<ipython-input-1-9b07313e2dc5>:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  training_spam = np.loadtxt(open("data/training_spam.csv"), delimiter=",").astype(np.int)


Your training set consists of 1000 rows and 55 columns. Each row corresponds to one email message. The first column is the _response_ variable and describes whether a message is spam `1` or ham `0`. The remaining 54 columns are _features_ that you will use to build a classifier. These features correspond to 54 different keywords (such as "money", "free", and "receive") and special characters (such as ":", "!", and "$"). A feature has the value `1` if the keyword appears in the message and `0` otherwise.

As mentioned there is also a 500 row set of *test data*. It contains the same 55 columns.

In [2]:
testing_spam = np.loadtxt(open("data/testing_spam.csv"), delimiter=",").astype(np.int)
print("Shape of the spam testing data set:", testing_spam.shape)
print(testing_spam)

Shape of the spam testing data set: (500, 55)
[[1 0 0 ... 1 1 1]
 [1 1 0 ... 1 1 1]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 1 0 0]]


<ipython-input-2-7b09f440222b>:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  testing_spam = np.loadtxt(open("data/testing_spam.csv"), delimiter=",").astype(np.int)


## Part One
Write all of the code for your classifier below this cell. There is some very rough skeleton code in the cell directly below. You may insert more cells below this if you wish, but you must not duplicate any cells as this can break the grading script.

### Submission Requirements
Your code must provide a variable with the name `classifier`. This object must have a method called `predict` which takes input data and returns class predictions. The input will be a single $n \times 54$ numpy array, your classifier should return a numpy array of length $n$ with classifications. There is a demo in the cell below, and a test you can run before submitting to check your code is working correctly.

Your code must run on our test machine in under 30 seconds. If you wish to train a more complicated model (e.g. neural network) which will take longer, you are welcome to save the model's weights as a file and then load these in the cell below so we can test it. You must include the code which computes the original weights, but this must not run when we run the notebook – comment out the code which actually executes the routine and make sure it is clear what we need to change to get it to run. Remember that we will be testing your final classifier on additional hidden data.

In [3]:
# This skeleton code simply classifies every input as ham
#
# Here you can see there is a parameter k that is unused, the
# point is to show you how you could set up your own. You might
# also pass in extra data via a train method (also does nothing
# here). Modify this code as much as you like so long as the 
# accuracy test in the cell below runs.

class SpamClassifier:
    def __init__(self, k):
        self.k = k
        self.y = 0
        
    def train(self):
        self.k = self.estimate_log_class_priors(training_spam)
        self.y = self.estimate_log_class_conditional_likelihoods(training_spam)
        pass
    
    #Returns the logged values of class priors given the a provided labelled dataset.
    def estimate_log_class_priors(self,data):
        zero = 0
        one = 0
        
        for counter in range(0,data.shape[0],1):
            temp = data[counter][0]
            if temp == 1:
                one+=1
            elif temp == 0:
                zero+=1
        #Turning these values into proportions
        zero = zero/data.shape[0]
        one = one/data.shape[0]

        #Calculating the logarithm of the proportion of 0s and 1s.
        #log(p(C=0))
        ECP0 = np.log(zero)

        #log(p(C=1))
        ECP1 = np.log(one)

        #Appending to numpy array
        log_class_priors = np.array([ECP0, ECP1])

        return log_class_priors
    
    #Returns the logged likelihoods of particular words being spam or ham
    def estimate_log_class_conditional_likelihoods(self,data, alpha=1.0):
    
        a = np.array([])
        b = np.array([])
        aa = np.array([])
        bb = np.array([])

        ##if zero, then when class is zero - check for 1s and append to relevant array
        ## if one, the when class is one - check for 1s and append to relevant array

        #Creating a nunmpy array to be returned
        for x in range(1,data.shape[1],1):
            oneA = 0
            oneB = 0
            oneBTotal = 0
            oneATotal = 0
            k = data.shape[1]
            for y in range(0,data.shape[0],1):
                for count in range(0,2,1):
                    temp = data[y][x]
                    if data[y][0] == 1:
                        if temp == 1:
                            oneA+=1 
                    if data[y][0] == 0:
                        if temp == 1:
                            oneB+=1

            a = np.append(a,oneA)
            b = np.append(b,oneB)

            oneATotal += oneA
            oneBTotal += oneB
        
        #Applying laplace smoothn if alpha > 0
        if alpha > 0:
            for n in range(0,a.size,1):
                numeratorA = a[n] + 1
                denominatorA = oneATotal + k
                result = numeratorA / denominatorA
                aa = np.append(aa,np.log(result))

            for m in range(0,b.size,1):
                numeratorB = b[m] +1
                denominatorB = oneBTotal + k
                result = numeratorB / denominatorB
                bb = np.append(bb,np.log(result))
        else:
            for n in range(0,a.size,1):
                numeratorA = a[n]
                denominatorA = oneATotal
                if numeratorA != 0:
                    if denominatorA !=0:
                        result = numeratorA / denominatorA
                        aa = np.append(aa,np.log(result))
                    else:
                        aa = np.append(aa,0)  
                else:
                    aa = np.append(aa,0) 

            for m in range(0,b.size,1):
                numeratorB = b[m]
                denominatorB = oneBTotal
                if numeratorB != 0:
                    if denominatorB !=0:
                        result = numeratorB / denominatorB
                        bb = np.append(bb,np.log(result))
                    else:
                        bb = np.append(bb,0)  
                else:
                    bb = np.append(bb,0) 



        theta = np.array([aa,bb])
        return theta

    #Given binary data set and having trained on other data, predict whether the data is ham or spam    
    def predict(self, new_data):
            #def predict(new_data, log_class_priors, log_class_conditional_likelihoods):
        log_class_priors = self.k
        log_class_conditional_likelihoods = self.y
        class_predictions = np.array([])
        
        #Determine likelihood of given data being spam or ham
        for y in range(0,new_data.shape[0],1):
            equaliser = 0
            for x in range(0,new_data.shape[1],1):
                #If value in row is one
                temp = new_data[y][x]
                if temp == 1:
                    spam = log_class_conditional_likelihoods[0][x]+log_class_priors[0]
                    ham = log_class_conditional_likelihoods[1][x]+log_class_priors[1]
                    if spam > ham:
                        equaliser +=1
                    else:
                        equaliser -=1.2
                ##This
                else:
                    equaliser-=0.01
            if equaliser > 1:
                class_predictions = np.append(class_predictions,1)
            else:
                class_predictions = np.append(class_predictions,0)

        #returns predictions
        return class_predictions

    

def create_classifier():
    classifier = SpamClassifier(k=1)
    classifier.train()
    #classifier.predict()
    return classifier

classifier = create_classifier()

### Accuracy Estimate
In the cell below there is a function called `my_accuracy_estimate()` which returns `0.5`. Before you submit the assignment, write your best guess for the accuracy of your classifier into this function, as a percentage between `0.5` and `1`. So if you think you will get 80% of inputs correct, return the value `0.8`. This will form a small part of the marking criteria for the assignment, to encourage you to test your own code without bias – give the *most accurate* estimate you can.

*Note* that there is no sense giving a value lower than `0.5` – if you are getting a score in this region, you can flip all of your predictions to get a better score!

In [4]:
def my_accuracy_estimate():
    return 0.878

Write all of the code for your classifier above this cell.

### Testing Details
Your classifier will be tested against some hidden data from the same source as the original. The accuracy (percentage of classifications correct) will be calculated, then benchmarked against common methods. At the very high end of the grading scale, your accuracy will also be compared to the best submissions from other students (in your own cohort and others!). Your estimate from the cell above will also factor in, and you will be rewarded for being close to your actual accuracy (overestimates and underestimates will be treated the same).

#### Test Cell
The following code will run your classifier against the provided test data. To enable it, set the constant `SKIP_TESTS` to `False`.

The original skeleton code above classifies every row as ham, but once you have written your own classifier you can run this cell again to test it. So long as your code sets up a variable called `classifier` with a method called `predict`, the test code will be able to run. 

Of course you may wish to test your classifier in additional ways, but you *must* ensure this version still runs before submitting.

**IMPORTANT**: you must set `SKIP_TESTS` back to `True` before submitting this file!

In [7]:
SKIP_TESTS = False

def tests():
    testing_spam = np.loadtxt(open("data/testing_spam.csv"), delimiter=",").astype(np.int)
    test_data = testing_spam[:, 1:]
    test_labels = testing_spam[:, 0]

    predictions = classifier.predict(test_data)
    accuracy = np.count_nonzero(predictions == test_labels)/test_labels.shape[0]
    print(f"Accuracy on test data is: {accuracy}")
    
if not SKIP_TESTS:
    tests()

Accuracy on test data is: 0.886


<ipython-input-7-4ff6bf51d460>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  testing_spam = np.loadtxt(open("data/testing_spam.csv"), delimiter=",").astype(np.int)


## Submission Test
The following cell tests if your notebook is ready for submission. **You must not skip this step!**

Restart the kernel and run the entire notebook (Kernel → Restart & Run All). Now look at the output of the cell below. 

*If there is no output, then your submission is not ready.* Either your code is still running (did you forget to skip tests?) or it caused an error.

As previously mentioned, failing to follow these instructions can result in a grade of zero.

In [6]:
def submission_tests():
    import sys
    import pathlib

    fail = False;

    if not SKIP_TESTS:
        fail = True;
        print("You must set the SKIP_TESTS constant to True in the cell above.")

    p3 = pathlib.Path('./spamclassifier.ipynb')
    if not p3.is_file():
        fail = True
        print("This notebook file must be named spamclassifier.ipynb")

    if "create_classifier" not in globals():
        fail = True;
        print("You must include a function called create_classifier.")

    if "my_accuracy_estimate" not in globals():
        fail = True;
        print("You must include a function called my_accuracy_estimate which returns a hard-coded value between 0.5 and 1.")
    else:
        if my_accuracy_estimate() == 0.5:
            print("Warning:")
            print("You do not seem to have provided an accuracy estimate, it is set to 0.5.")
            print("This is the actually the worst possible accuracy – if your classifier")
            print("got 0.1 then it could invert its results to get 0.9!")
    print()

    if fail:
        sys.stderr.write("Your submission is not ready! Please read and follow the instructions above.")
    else:
        print("All checks passed. When you are ready to submit, upload the notebook to the")
        print("assignment page, without changing any filenames.")
        print()
        print("If you need to submit multiple files, you can archive them in a .zip file. (No other format.)")
        
submission_tests()

You must set the SKIP_TESTS constant to True in the cell above.



Your submission is not ready! Please read and follow the instructions above.

In [7]:
# This is a test cell. Please do not modify or delete.